In [ ]:
import h5py
import numpy as np

from ada import Assembly, Beam, Material, Part
from ada.fem import Bc, FemSet, Step
from ada.fem.meshing.gmshapiv2 import GmshSession, GmshOptions
from ada.fem.utils import get_beam_end_nodes
from ada.materials.metals import CarbonSteel

from ada.visualize.femviz import visualize_it

In [ ]:
def fundamental_eigenfrequency(bm: Beam):
    L = bm.length
    E = bm.material.model.E
    rho = bm.material.model.rho
    I = bm.section.properties.Iy
    return (1 / (2 * np.pi)) * (3.5156 / L ** 2) * np.sqrt(E * I / rho)


bm = Beam("MyBeam", (0, 0.5, 0.5), (5, 0.5, 0.5), "IPE400", Material("S420", CarbonSteel("S420")))
bm

In [ ]:
f1 = fundamental_eigenfrequency(bm)
f2 = 6.268 * f1
f3 = 17.456 * f1
print(f"1: {f1}\n2: {f2}\n3: {f3}")

In [ ]:
def make_fem(beam, assembly, part, geom_repr):
    print(geom_repr)
    with GmshSession(silent=True, options=GmshOptions(Mesh_ElementOrder=1)) as gs:
        gs.add_obj(bm, geom_repr=geom_repr)
        gs.mesh(0.1)
        part.fem = gs.get_fem()
    fix_set = part.fem.add_set(FemSet("bc_nodes", get_beam_end_nodes(beam), "nset"))
    assembly.fem.add_bc(Bc("Fixed", fix_set, [1, 2, 3, 4, 5, 6]))
    assembly.fem.add_step(Step("Eigen", "eigenfrequency"))


p = Part("MyPart")
a = Assembly("MyAssembly") / [p / bm]
make_fem(bm, a, p, "line")
res = a.to_fem("Cantilever_CA_EIG_bm", "code_aster", overwrite=True, execute=True)

In [ ]:
a

In [ ]:
visualize_it(res.results_file_path, temp_file="../_temp_bm.vtu")

In [ ]:
f = h5py.File(res.results_file_path)
modes = f.get("CHA/modes___DEPL")

for mname, m in modes.items():
    mode = m.attrs["NDT"]
    freq = m.attrs["PDT"]

    print(mode, freq)

In [ ]:
p = Part("MyPart")
a = Assembly("MyAssembly") / [p / bm]
make_fem(bm, a, p, "shell")
res = a.to_fem("Cantilever_CA_EIG_sh", "code_aster", overwrite=True, execute=True)

In [ ]:
visualize_it(res.results_file_path, temp_file="../_temp_sh.vtu")

In [ ]:
f = h5py.File(res.results_file_path)
modes = f.get("CHA/modes___DEPL")

for mname, m in modes.items():
    mode = m.attrs["NDT"]
    freq = m.attrs["PDT"]

    print(mode, freq)